# 利用VGG19对CIFAR10数据集进行分类

### VGG19起源与介绍
VGGNet是牛津大学计算机视觉组（Visual Geometry Group）和Google DeepMind公司的研究员一起研发的卷积神经网络。VGGNet探索了卷积神经网络的深度与其性能之间的关系，通过反复的使用$3\times3$的小型卷积核和$2\times2$的最大池化层，VGGNet成功地构筑了16～19层深的卷积神经网络。
如图所示，即为VGG19的网络结构：
![png](https://raw.githubusercontent.com/shiyadong123/Myimage/master/20170816092916647.png)

### VGG16与VGG19的直观对比
![png](https://github.com/shiyadong123/Myimage/blob/master/20190217165325787.png?raw=true)

### 预准备
在搭建VGG19网络结构之前，首先做预准备，包括：
+ 1.导入必要的库
+ 2.CIFAR10数据集的预处理
+ 3.定义训练模型用的辅助函数

In [5]:
# 1.导入必要的库
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.datasets as dsets
import torchvision.transforms as trans
import time

In [6]:
# 超参数
BATCH_SIZE = 100
nepochs = 50
LR = 0.001

# 定义损失函数为交叉熵损失 loss_func
loss_func = nn.CrossEntropyLoss()

# 可以在GPU或者CPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# 2.CIFAR10数据集的预处理

# CIFAR10的输入图片各channel的均值 mean 和标准差 std 
mean = [x/255 for x in [125.3, 23.0, 113.9]] 
std = [x/255 for x in [63.0, 62.1, 66.7]]
n_train_samples = 50000

# 如果是多进程需要加一个main函数，否则会报错
if __name__ == '__main__':
    
    # 数据增强-->训练集
    train_set = dsets.CIFAR10(root='CIFAR10/',  # 数据集保存路径
                              train=True,
                              download=False,   # 如果未下载，改为True；如果已经下载好，改为False
                              transform=trans.Compose([
                                 trans.RandomHorizontalFlip(),
                                 trans.RandomCrop(32, padding=4),
                                 trans.ToTensor(),
                                 trans.Normalize(mean, std)
                             ]))
    train_dl = DataLoader(train_set,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=6)        # 多进程
    
    # train_set.train_data = train_set.train_data[0:n_train_samples]
    # train_set.train_labels = train_set.train_labels[0:n_train_samples]
    
    # 测试集
    test_set = dsets.CIFAR10(root='CIFAR10/',   # 数据集保存路径
                             train=False,
                             download=False,    # 如果未下载，改为True；如果已经下载好，改为False
                             transform=trans.Compose([
                                trans.ToTensor(),
                                trans.Normalize(mean, std)
                            ]))

    test_dl = DataLoader(test_set,
                         batch_size=BATCH_SIZE,
                         num_workers=6)         # 多进程

In [15]:
# 3.定义训练的辅助函数，其中包括误差 error 与正确率 accuracy
def eval(model, loss_func, dataloader):

    model.eval()
    loss, accuracy = 0, 0
    
    # torch.no_grad显示地告诉pytorch，前向传播的时候不需要存储计算图
    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            logits = model(batch_x)
            error = loss_func(logits, batch_y)
            loss += error.item()

            probs, pred_y = logits.data.max(dim=1)
            accuracy += (pred_y==batch_y.data).float().sum()/batch_y.size(0)

    loss /= len(dataloader)
    accuracy = accuracy*100.0/len(dataloader)
    return loss, accuracy


def train_epoch(model, loss_func, optimizer, dataloader):

    model.train()
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        logits = model(batch_x)
        error = loss_func(logits, batch_y)
        error.backward()
        optimizer.step()

### VGG19网络结构
![png](https://github.com/shiyadong123/Myimage/blob/master/68747470733a2f2f6c6968616e2e6d652f6173736574732f696d616765732f7667672d6865726f2d636f7665722e6a7067.jpg?raw=true)

根据上图的VGG19网络结构，开始正式搭建VGG19模型，为了方便起见，先定义卷积层

In [10]:
# 定义卷积层，在VGGNet中，均使用3x3的卷积核
def conv3x3(in_features, out_features): 
    return nn.Conv2d(in_features, out_features, kernel_size=3, padding=1)

In [11]:
# 搭建VGG19，除了卷积层外，还包括2个全连接层（fc_1、fc_2），1个softmax层
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            # 1.con1_1
            conv3x3(3, 64),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # 2.con1_2
            conv3x3(64, 64),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 3.con2_1
            conv3x3(64, 128),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # 4.con2_2
            conv3x3(128, 128),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 5.con3_1
            conv3x3(128, 256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 6.con3_2
            conv3x3(256, 256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 7.con3_3
            conv3x3(256, 256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            # 8.con3_4
            conv3x3(256, 256),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 9.con4_1
            conv3x3(256, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 10.con4_2
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 11.con4_3
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 12.con4_4
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 13.con5_1
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 14.con5_2
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 15.con5_3
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            # 16.con5_4
            conv3x3(512, 512),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2),
            )

        self.classifier = nn.Sequential(
            # 17.fc_1
            nn.Linear(512, 4096),
            nn.ReLU(),
            nn.Dropout(),
            # 18.fc_2
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            # 19.softmax
            nn.Linear(4096, 10),  # 最后通过softmax层，输出10个类别
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

定义好VGG19网络之后，开始训练

In [17]:
vgg19 = VGG().to(device)
# 可以通过打印vgg19观察具体的网络结构
# print(vgg19) 

# 使用Adam进行优化处理
optimizer = torch.optim.Adam(vgg19.parameters(), lr=LR)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1)
learn_history = []

In [18]:
print('开始训练VGG19……')

for epoch in range(nepochs):
    # 训练开始时间
    since = time.time()
    train_epoch(vgg19, loss_func, optimizer, train_dl)
    
    # 每训练5轮输出一次结果
    if (epoch)%5 == 0:
        tr_loss, tr_acc = eval(vgg19, loss_func, train_dl)
        te_loss, te_acc = eval(vgg19, loss_func, test_dl)
        learn_history.append((tr_loss, tr_acc, te_loss, te_acc))
        # 完成一批次训练的结束时间
        now = time.time()
        print('[%3d/%d, %.0f seconds]|\t 训练误差: %.1e, 训练正确率: %.2f\t |\t 测试误差: %.1e, 测试正确率: %.2f'%(
            epoch+1, nepochs, now-since, tr_loss, tr_acc, te_loss, te_acc))

开始训练VGG19……
[  1/50, 136 seconds]|	 tr_err: 1.9e+00, tr_acc: 24.74	 |	 te_err: 1.9e+00, te_acc: 24.67
[  6/50, 138 seconds]|	 tr_err: 8.4e-01, tr_acc: 71.55	 |	 te_err: 8.7e-01, te_acc: 70.88
[ 11/50, 137 seconds]|	 tr_err: 4.8e-01, tr_acc: 84.10	 |	 te_err: 5.4e-01, te_acc: 82.29
[ 16/50, 136 seconds]|	 tr_err: 3.5e-01, tr_acc: 88.74	 |	 te_err: 4.7e-01, te_acc: 85.54
[ 21/50, 136 seconds]|	 tr_err: 2.7e-01, tr_acc: 91.51	 |	 te_err: 4.1e-01, te_acc: 87.08
[ 26/50, 136 seconds]|	 tr_err: 3.1e-01, tr_acc: 90.48	 |	 te_err: 5.1e-01, te_acc: 85.43
[ 31/50, 136 seconds]|	 tr_err: 1.7e-01, tr_acc: 94.52	 |	 te_err: 4.0e-01, te_acc: 88.79
[ 36/50, 136 seconds]|	 tr_err: 1.5e-01, tr_acc: 95.37	 |	 te_err: 4.1e-01, te_acc: 88.97
[ 41/50, 136 seconds]|	 tr_err: 1.8e-01, tr_acc: 94.19	 |	 te_err: 4.2e-01, te_acc: 87.44
[ 46/50, 136 seconds]|	 tr_err: 1.1e-01, tr_acc: 96.65	 |	 te_err: 4.1e-01, te_acc: 89.16


根据输出的结果，我们可以很明显的看出，在训练轮次增加后，正确率有了明显的提高，训练完50轮后，测试集的正确率达到89.16%，如果我们进一步增加训练轮次，正确率应该还会更高。